<a href="https://colab.research.google.com/github/Horese07/Elhoucine_Elachguar_ADIA-G2-Java/blob/main/ProjetCNN_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

                          
<center><h1><a><strong> Projet de Classification de Chiffres Manuscrits avec CNN<strong></a> </h1></center>
<fieldset>
<center><h2> : Classification de Chiffres Manuscrits avec Réseaux de Neurones Convolutifs
(CNN) sur le jeu de données MNIST</h2></center>
</fielset>



#<center><h1><strong><a>I.Introduction : </a></strong>Contexte et Objectifs</h1></center>
  

<h3><i>Contexte:</i><h3>

 <p>La reconnaissance de chiffres manuscrits est un problème classique de la vision par ordinateur et de l'apprentissage automatique. Ce problème consiste à identifier des chiffres écrits à la main, ce qui présente plusieurs défis en raison des variations dans le style d'écriture de différentes personnes. Le jeu de données MNIST (Modified National Institute of Standards and Technology) est largement utilisé pour ce type de tâches. Il contient 70,000 images en niveaux de gris de 28x28 pixels représentant les chiffres de 0 à 9, avec 60,000 images destinées à l'entraînement et 10,000 images pour le test.</p>


 <h3><i>Objectif:</i><h3>
<p>L'objectif de ce projet est de construire un modèle de réseau de neurones convolutifs (CNN) capable de classifier les chiffres manuscrits du jeu de données MNIST avec une haute précision. Pour atteindre cet objectif, les étapes suivantes seront suivies :

1.   <strong>Prétraitement des Données :</strong> Préparer les données en les normalisant et en appliquant le one-hot encoding aux labels.
2.   <strong>Construction du Modèle CNN :</strong> Construire une architecture CNN adaptée à la classification d'images de chiffres manuscrits.
3.   <strong>Entraînement et Évaluation du Modèle :</strong> Entraîner le modèle sur les données d'entraînement et l'évaluer sur les données de validation et de test.
4.   <strong>Visualisation des Résultats :</strong> Analyser les performances du modèle à travers des graphiques et des courbes d'apprentissage.
5.<strong>Améliorations et Expérimentations :</strong> Expérimenter avec des techniques d'amélioration comme l'augmentation de données pour améliorer la robustesse et la précision du modèle.








#<center> <h2><strong><a>II.Prétraitement des Données<a></strong></h2></center>

<h3><i>1. Importation des Bibliothèques<i></h3>

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2
import os
from skimage.transform import  rotate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Create fucnction for add and convert images to gray
def check_and_convert_to_grayscale(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if image loading was successful
    if image is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")

    # Check the number of channels in the image
    if len(image.shape) == 3 and image.shape[2] == 3:
        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    return gray_image

def preprocess_image(image_path, img_size=(28, 28)):
    # Convert to grayscale and resize
    image = check_and_convert_to_grayscale(image_path)
    image_resized = cv2.resize(image, img_size)

    # Normalize the image
    image_normalized = image_resized / 255.0

    return image_normalized

def load_my_images(directory, img_size=(28, 28)):
    images = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            label = int(filename.split('_')[0])  # like '3_image1.jpg' donc label = 3
            image_path = os.path.join(directory, filename)
            image = preprocess_image(image_path, img_size)
            images.append(image)
            labels.append(label)

    return np.array(images), np.array(labels)


<h3><i>2. Chargement et Prévisualisation des Données</i></h3>

First We will add our data

In [ ]:
my_images, my_labels = load_my_images('/content/drive/MyDrive/Colab Notebooks/Numbers')



# Visualiser quelques exemples d'images
plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(my_images[i], cmap=plt.cm.binary)
    plt.xlabel(my_labels[i])
plt.show()

Right now we Are going to augment the our data

In [ ]:
# Create a new array to store the augmented images
augmented_images = []
augmented_labels = []

for image, label in zip(my_images, my_labels):
    # Generate a random angle for rotation
    random_angle = np.random.randint(low=0, high=60)
    # Rotate the image
    rotated_image = rotate(image, angle=random_angle)
    # Convert the rotated image to a NumPy array
    rotated_image = np.array(rotated_image)
    rotated_lablel = np.array(label)
    # Append the rotated image to augmented_images
    augmented_images.append(rotated_image)
    augmented_labels.append(rotated_lablel)

# Convert augmented_images to numpy array
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Add augmented images to the original my_images array
my_images = np.concatenate((my_images, augmented_images))
my_labels = np.concatenate((my_labels, augmented_labels))

# Visualize the rotated images
fig = plt.figure(figsize=(10, 10))
for i in range(4):
    ax = fig.add_subplot(2, 2, i+1)
    ax.imshow(augmented_images[i])  # Visualizing augmented images
    title = "Rotated image {}".format(i+1, augmented_labels[i])
    ax.set_title(title)

plt.show()

after that we add our data to the data from mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Ajouter les images augmentées à train_images
train_images = np.concatenate((train_images, my_images))
train_labels = np.concatenate((train_labels, my_labels))

# Visualiser quelques exemples d'images
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()


Le jeu de données MNIST contient des images manuscrites de chiffres allant de 0 à 9. Il y a donc 10 classes dans ce dataset, chaque classe correspondant à un chiffre distinct :


0.  – Chiffre 0
1.  – Chiffre 1
2.  – Chiffre 2
3.  – Chiffre 2
4.  – Chiffre 4
5.  – Chiffre 5
6.   – Chiffre 6
7.   – Chiffre 7
8.   – Chiffre 8
9.   – Chiffre 9


<h3><i>3. Prétraitement des Données</i><h3>

In [ ]:
#Normalisation des Images
# Les valeurs de pixels vont de 0 à 255. En divisant par 255, on normalise ces
# valeurs pour qu'elles soient dans la plage [0, 1].
train_images = train_images / 255.0
test_images = test_images / 255.0


In [ ]:
#One-Hot Encoding des Labels
# Convertir les labels des chiffres en un format binaire (one-hot encoding) pour
# qu'ils soient compatibles avec la couche de sortie du modèle CNN.
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [ ]:
#Reshape des Images
# Adapter la forme des images pour qu'elles soient compatibles avec
# l'architecture CNN, qui attend des images avec une dimension de canal
# supplémentaire.
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))


In [ ]:
# Division des données en ensemble d'entraînement et de validation
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)


#<center> <h2><strong><a>III.Construction du Modèle CNN</a></strong></h2></center>


*  <strong>Convolution 2D :</strong> Applique un filtre de convolution sur l'image pour extraire des caractéristiques (features).
*  <strong> ReLU : </strong>Fonction d'activation rectifiée qui introduit la non-linéarité.
* <strong> MaxPooling : </strong>Réduit les dimensions spatiales de l'image, conservant les informations les plus importantes.
*   <strong>Flatten :</strong> Aplatie les matrices en un vecteur.
*<strong>  Dense : </strong>Couche entièrement connectée qui apprend les combinaisons non linéaires de caractéristiques.


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


#<center> <h2><strong><a>IV.Entraînement et Évaluation du Modèle </a></strong></h2></center>


*  <strong> Adam Optimizer :</strong> Algorithme d'optimisation adaptatif qui ajuste les taux d'apprentissage pour chaque paramètre.
*  <strong> Categorical Crossentropy :</strong> Fonction de perte utilisée pour les classifications multi-classes.

* <strong> Metrics : </strong>accuracy mesure la proportion de prédictions correctes.



In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=5,
                    validation_data=(val_images, val_labels))


In [ ]:
pred = model.predict(test_images[7].reshape(1, 28, 28, 1))
label = np.argmax(pred)
print(label)

# <center><h2><strong><a>V.Évaluation et Analyse du Modèle</a></strong></h2></center>

<p>Les courbes d'apprentissage montrent l'évolution de l'exactitude pendant l'entraînement et la validation, aidant à identifier des problèmes de surapprentissage ou de sous-apprentissage.<p>

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc}')

# Visualisation des courbes d'apprentissage
losses = pd.DataFrame(history.history)
losses[['accuracy', 'val_accuracy']].plot()
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()




La courbe d'accuracy en fonction des epochs montre que le modèle CNN est bien entraîné sur le jeu de données MNIST. L'accuracy atteint un plateau après quelques epochs, indiquant que le modèle a appris les caractéristiques essentielles des données sans surapprendre. La courbe stable et élevée de l'accuracy de validation confirme la capacité du modèle à généraliser correctement aux nouvelles données.

In [ ]:
losses[['loss', 'val_loss']].plot()
plt.title('Loss vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

La courbe de perte montre que votre modèle CNN est bien entraîné sur le jeu de données MNIST. La perte d'entraînement diminue régulièrement, indiquant une bonne progression de l'apprentissage. La perte de validation reste basse avec des fluctuations modérées, suggérant que le modèle généralise bien aux nouvelles données, avec une légère tendance au surajustement dans les dernières epochs.

#<center> <h2><strong><a>VI.Améliorations et Expérimentations</a></strong></h2><center>

<i>L'augmentation de données génère des variantes des images d'entraînement en appliquant des transformations aléatoires comme des rotations, des zooms et des translations. Cela aide à améliorer la généralisation du modèle en le rendant plus robuste aux variations des données.<i>

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(train_images)

In [ ]:
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    epochs=10,
                    validation_data=(val_images, val_labels))

In [ ]:
# Générer un batch d'images augmentées
augmented_images, augmented_labels = next(datagen.flow(train_images, train_labels, batch_size=25))

plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(augmented_images[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(np.argmax(augmented_labels[i]))
plt.show()

#<center> <h2><strong><a>VII.Évaluation sur les Données de Test</a></strong></h2><center>

<p>Formule : Precision = TP / (TP + FP)</p>
<p>Formule : recall   = TP / (TP + FN)</p>
<p>Formule : f1-Score = 2*(Recall * Precision)/(Recall + Precsion)</p>

In [ ]:
# Évaluer la performance sur les données de test
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc}')

# Prédictions sur les données de test
predictions = model.predict(test_images)

# Convertir les prédictions et les labels de test en labels de classe
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(test_labels, axis=1)

# Afficher un rapport de classification
print(classification_report(true_labels, predicted_labels))

# Afficher une matrice de confusion
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print(conf_matrix)
plt.figure(figsize=(8, 8))
sns.heatmap(conf_matrix , annot = True)

In [ ]:
# Visualisation des prédictions pour quelques exemples
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])

    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i].reshape(28,28), cmap=plt.cm.binary)
    plt.xlabel(f"True: {true_labels[i]}, Pred: {predicted_labels[i]}")
plt.show()


<h3><i>Creation d'un interface web en utilisant  FLASK</i></h3>

In [ ]:
model.save('model.h5')

In [ ]:
!pip install flask pyngrok tensorflow pillow

In [ ]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import io
import base64

app = Flask(__name__)
model = load_model('model.h5')

def preprocess_image(image, target_size):
    if image.mode != "L":
        image = image.convert("L")
    image = image.resize(target_size)
    image = np.expand_dims(image, axis=0)
    image = np.expand_dims(image, axis=-1)
    image = image.astype("float32") / 255.0
    return image

@app.route('/')
def index():
    return '''
 <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Draw a Digit</title>
    <style>
        @import url("https://fonts.googleapis.com/css2?family=Fredoka:wght@400;600&display=swap");

        body {
            background-image: url('https://media.istockphoto.com/id/1470617656/vector/ai-artificial-intelligence-chipset-on-circuit-board-in-futuristic-concept-suitable-for.jpg?s=612x612&w=0&k=20&c=_wC-pphyNI2muaUHG4N9JuYXxJEMDuzx56Dvzr8ZDUk=');
            background-size: cover;
            background-repeat: no-repeat;
            background-position: center;
            font-family: 'Fredoka', sans-serif;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }

        .card {
            background-color: #ffffff;
            border-radius: 30px;
            text-align: center;
            padding: 15px;
            max-width: 475px;
            margin: 20px;
            box-shadow: 20px 20px 20px rgba(0, 0, 0, 0.1), -20px -20px 20px rgba(255, 255, 255, 0.7);
        }

        .illustration {
            margin: 30px;
        }

        .illustration lottie-player {
            width: 200px;
            height: 200px;
            border-radius: 100%;
            overflow: hidden;
            margin: 0 auto;
            box-shadow: 20px 20px 20px rgba(0, 0, 0, 0.1), -20px -20px 20px rgba(255, 255, 255, 0.7);
        }

        h1 {
            font-size: 2rem;
            line-height: 2.2rem;
            color: #2e2e2e;
            font-weight: bold;
            margin: 30px 0;
        }

        p {
            font-size: 1rem;
            line-height: 1.3rem;
            color: #7e7e7e;
            margin: 30px auto;
            max-width: 80%;
        }

        button {
            font-size: 1.1rem;
            font-weight: bold;
            padding: 15px 60px;
            border-radius: 25px;
            color: white;
            border: 0;
            margin: 30px 0;
            outline: none;
            background-color: #81c784;
            box-shadow: 10px 10px 10px rgba(0, 0, 0, 0.1), -10px -10px 10px rgba(255, 255, 255, 0.7);
            transition: ease all 0.3s;
            cursor: pointer;
        }

        button:hover {
            box-shadow: 20px 20px 20px rgba(0, 0, 0, 0.1), -20px -20px 20px rgba(255, 255, 255, 0.7);
            transform: translateY(-3px);
        }

        button:active {
            transform: scale(0.9);
            box-shadow: inset -1px -1px 3px rgba(0, 0, 0, 0.2), -20px -20px 20px rgba(255, 255, 255, 0.7);
        }

        canvas {
            border: 2px solid #000000;
            margin: 20px 0;
        }
    </style>
</head>
<body>
    <div class="card">
        <div class="illustration">
            <lottie-player src="https://assets10.lottiefiles.com/packages/lf20_LrcfNr.json" background="white" speed="1" loop autoplay></lottie-player>
        </div>
        <h1>Dessiner un nombre entre 0 et 9</h1>
        <canvas id="canvas" width="280" height="280"></canvas>
        <br>
        <button onclick="clearCanvas()">Clear</button>
        <button onclick="predict()">Predict from Drawing</button>
        <p id="resultDrawing"></p>
    </div>

    <div class="card">
        <h1>Prediction à partir d'une image</h1>
        <input type="file" id="fileInput" accept="image/*">
        <br>
        <button onclick="predictImage()">Predict from Image</button>
        <p id="resultImage"></p>
    </div>

    <script>
        var canvas = document.getElementById('canvas');
        var ctx = canvas.getContext('2d');
        ctx.fillStyle = "black";
        ctx.fillRect(0, 0, canvas.width, canvas.height);

        var drawing = false;

        canvas.addEventListener('mousedown', function(e) {
            drawing = true;
            draw(e);
        });

        canvas.addEventListener('mouseup', function() {
            drawing = false;
            ctx.beginPath();
        });

        canvas.addEventListener('mousemove', draw);

        canvas.addEventListener('touchstart', function(e) {
            drawing = true;
            draw(e.touches[0]);
        });

        canvas.addEventListener('touchend', function() {
            drawing = false;
            ctx.beginPath();
        });

        canvas.addEventListener('touchmove', function(e) {
            draw(e.touches[0]);
        });

        function draw(e) {
            if (!drawing) return;
            ctx.lineWidth = 10;
            ctx.lineCap = 'round';
            ctx.strokeStyle = 'white';

            ctx.lineTo(e.clientX - canvas.offsetLeft, e.clientY - canvas.offsetTop);
            ctx.stroke();
            ctx.beginPath();
            ctx.moveTo(e.clientX - canvas.offsetLeft, e.clientY - canvas.offsetTop);
        }

        function clearCanvas() {
            ctx.fillStyle = "black";
            ctx.fillRect(0, 0, canvas.width, canvas.height);
            ctx.beginPath();
        }

        function predict() {
            var dataURL = canvas.toDataURL('image/png');
            fetch('/predict', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ image: dataURL })
            })
            .then(response => response.json())
            .then(data => {
                document.getElementById('resultDrawing').textContent = 'Prediction: ' + data.prediction;
            })
            .catch(error => {
                console.error('Error:', error);
            });
        }

        function predictImage() {
            var fileInput = document.getElementById('fileInput');
            var file = fileInput.files[0];
            var reader = new FileReader();

            reader.onload = function(event) {
                var dataURL = event.target.result;
                fetch('/predict', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ image: dataURL })
                })
                .then(response => response.json())
                .then(data => {
                    document.getElementById('resultImage').textContent = 'Prediction: ' + data.prediction;
                })
                .catch(error => {
                    console.error('Error:', error);
                });
            };

            reader.readAsDataURL(file);
        }
    </script>
</body>
</html>
'''
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    image_data = data["image"]
    image_data = image_data.split(",")[1]
    image = base64.b64decode(image_data)
    image = Image.open(io.BytesIO(image))
    processed_image = preprocess_image(image, target_size=(28, 28))

    prediction = model.predict(processed_image).argmax(axis=1)[0]
    response = {"prediction": int(prediction)}

    return jsonify(response)



In [ ]:
!ngrok authtoken  2gkZNiq716dFIEXPmP2mAGShvYG_5SDQGJiHkv3cQxzGnEFNL

In [ ]:
from pyngrok import ngrok

# Démarrer le tunnel ngrok
tunnel = ngrok.connect(5000)
public_url = tunnel.public_url
print(" * ngrok tunnel: " + public_url)

# Démarrer l'application Flask
app.run()
